In [1]:
import minsearch
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Index Documents

In [4]:
index = minsearch.Index(
        text_fields=["question", "text", "section"],  #use for performant search, look for text that can help answer the question
        keyword_fields=["course"]  #which documents to look in
        )

In [5]:
index.fit(documents)

In [6]:
q = "the course has already started, can i still enroll?"

In [7]:
boost = {"question": 3.0, "section": 0.5} # if a section of document has relevant terms, attirbute more importance to those sections. For example, question field is 3x more important than text field

results = index.search(
    query = q,
    boost_dict=boost,
    num_results = 5
    )

In [8]:
results_de_zoom = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results = 5
)

results_de_zoom

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### Use GPT-4o model for generic response

In [ ]:
from openai import OpenAI


In [11]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('API_KEY')

In [12]:
client = OpenAI()

In [13]:
q = "the course has already started, can i still enroll?"
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

In [14]:
#generic response with 0 context
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the institution or platform offering the course. Here are a few steps you can take:\n\n1. **Check the Course Platform:** Look for any specific information about late enrollment on the course's webpage or platform.\n\n2. **Contact the Instructor or Administrator:** Reach out directly to the course instructor or the administrative office to inquire if late enrollment is possible.\n\n3. **Understand the Policies:** Some institutions have a grace period or allow late enrollment if the course duration is long enough to accommodate catching up.\n\n4. **Consider Course Progress and Content:** Evaluate how much content you have missed and whether you can realistically catch up.\n\n5. **Look for Alternatives:** If enrollment in this particular course isn't possible, check if there are future sessions or similar courses available.\n\nIf you're serious about enrolling, it's worth reaching out directly to get a definitiv

### Use GPT-4o model for specific response

In [16]:
context = ""
for doc in results: 
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [17]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question. If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {q} 

CONTEXT: {context}

""".strip()

In [18]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt_template}]
)

response.choices[0].message.content

"Yes, you can still enroll even after the course has started. However, you may not be able to submit some of the homework assignments. In order to receive a certificate, you must submit 2 out of 3 course projects and review 3 peers' projects by their respective deadlines."

### Modularize the code

In [19]:
def search(query):
    boost = {"question": 3.0, "section": 0.5} 

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 5
        )
    
    return results

In [20]:
def build_prompt(query, search_results):
    
    context = ""
    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    
    prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.

QUESTION: {query} 

CONTEXT: {context}

""".strip()
    
    return prompt

In [21]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

    return response.choices[0].message.content 

In [22]:
query = "how do i run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results )
    return llm(prompt=prompt)

In [23]:
rag(query)

'To run Kafka in a terminal, navigate to your project directory and use the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file.'

In [24]:
rag('the course has already started, can i still enroll?')

'Yes, you can still join the course after the start date. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects, as you should not leave everything for the last minute.'

### Using Elasticsearch

In [25]:
from elasticsearch import Elasticsearch

In [27]:
es_client = Elasticsearch('http://localhost:9200')

In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [29]:
from tqdm.auto import tqdm

/workspaces/llms/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc) # create a record in the index for each document

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:04<00:00, 204.46it/s]


In [32]:
query = 'I just discovered the course. Can I still join it?'

In [33]:
# querying the data in the index

def elastic_search(query):

    search_query = {
        "size": 5, #how many index records to return
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #boost is determined by exponent ^ 
                        "type": "best_fields"
                    }
                },
                "filter": {   # basically a where caluse 
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs



In [34]:
def rag_elastic(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query=query, search_results=search_results )
    return llm(prompt=prompt)

In [35]:
rag_elastic(query=query)

'Yes, you can still join the course even if you have discovered it after the start date. You are still eligible to submit the homework without needing to register, but make sure to keep track of deadlines for the final projects as they are fixed.'